# Constants


In [5]:

import pandas as pd
import os
import shutil
from os import path as osp

import torch
from tiatoolbox.utils.misc import select_device
import random
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
import json
import glob

# from src.intensity import add_features_and_create_new_dicts

from src.featureextraction import get_cell_features, add_features_and_create_new_dicts
from src.train import stratified_split, recur_find_ext, run_once, rm_n_mkdir ,reset_logging
from src.graph_construct import create_graph_with_pooled_patch_nodes, get_pids_labels_for_key


ON_GPU = False
device = select_device(on_gpu=ON_GPU)

SEED = 5
random.seed(SEED)
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
# STAIN = 'BCL2'
# STAIN = 'HE'

FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
PATCH_SIZE = 224
OUTPUT_SIZE = PATCH_SIZE*8

WORKSPACE_DIR = Path(BASEDIR)
# GRAPH_DIR = WORKSPACE_DIR / f"graphs{STAIN}" 
# LABELS_PATH = WORKSPACE_DIR / "graphs/0_labels.txt"


# Graph construction
# PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500

LABEL_TYPE = 'multilabel' #'OS' #

GRAPHSDIR = Path(f'{BASEDIR}/graphs/{STAIN}')
LABELSPATH = f'{BASEDIR}/graphs/{STAIN}_labels.json'

NUM_EPOCHS = 100
NUM_NODE_FEATURES = 128
NCLASSES = 3

TRAIN_DIR = WORKSPACE_DIR / "training"
SPLIT_PATH = TRAIN_DIR / f"splits_{STAIN}.dat"
# RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{STAIN}_{datetime.now().strftime('%m_%d_%H_%M')}"


# Extract features: 20 + 4 minutes

In [ ]:
# read annotation csv, filter, and process intensity features
df = pd.read_csv(ANNPATH)

df = df[df['stain'] == STAIN]
df['area'] = (df['xe'] - df['xs']) *  (df['ye'] - df['ys'])/10000
df = df[df['area'] >= 150]  
df = df[df['xs']  >=0 ]
df = df[df['ys']  >=0 ]
df = df[df['xe']  >=0 ]
df = df[df['ye']  >=0 ]

df = df.reset_index()
##########

###############
# add intensity features
start_index = 0
end_index = len(df.index)

datpaths = []
imgpaths = []
updatpaths = []

# for index in range(start_index, 1):
for index in range(start_index, end_index):
    df_index = df['index'][index]
    patient_id = df['patient_id'][index]
    stain = df['stain'][index]
    tma_id = df['tma_id'][index]
    unique_id = str(patient_id) + '_' + stain + '_' + str(df_index)

    img_file_name = f"{FIDIR}/images/{patient_id}/{patient_id}_{stain}_{tma_id}_{OUTPUT_SIZE}_{df_index}.png"
    dat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/0.dat"
    updat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/{unique_id}.dat"

    datpaths.append(dat_file_name)
    imgpaths.append(img_file_name)
    updatpaths.append(updat_file_name)
    


In [ ]:
updatpaths[:3]

In [ ]:
# 16 minutes
# shutil.rmtree(FEATSDIR)
# if not osp.exists(FEATSDIR):
#     os.makedirs(FEATSDIR)
add_features_and_create_new_dicts(datpaths, imgpaths, updatpaths)

In [ ]:
# 4 minutes
gns = StandardScaler()
for featpath in tqdm(updatpaths):
    try:
        celldatadict = joblib.load(featpath)
        cellsfeats = np.array([v['intensity_feats'] for k, v in celldatadict.items()])
        gns.partial_fit(cellsfeats)
    except:
        print(featpath)

np.savez(FEATSCALERPATH, mean=gns.mean_, var=gns.var_)

# dd = np.load(FEATSCALERPATH)
# print(dd['mean'], gns.mean_)
# print(dd['var'], gns.var_)

In [ ]:
FEATSCALERPATH

# Prepare graphs : 4 minutes

In [2]:
from src.graph_construct import get_pids_multilabels_for_key

df = pd.read_csv(CLINPATH)
df =df.fillna(0)
# display(df.info())

def get_pids_multilabels_for_key(df, key_list, nclasses=3, idkey='patient_id'):
    df = df[[idkey]+ key_list]

    for key in key_list:
        if key == "OS":
            ckey = key+'_class'
            df[ckey] = int(0)
            separators = np.linspace(0, 1, nclasses+1)[0:-1]
            for isep, sep in enumerate(separators):
                sepval = df[key].quantile(sep)
                sepmask = df[key] > sepval
                df.loc[sepmask, ckey] = isep
        elif key in ['MYC IHC', 'BCL2 IHC', 'BCL6 IHC']:
            ckey = key+'_class'
            df[ckey] = int(0)
            df[ckey][df[key] >= 40] = 1
            df[ckey][df[key] < 40] = 0
        else:
            ckey = key+'_class'
            df[ckey] = df[key]
    return df

if LABEL_TYPE == 'multilabel':
    df_labels = get_pids_multilabels_for_key(df, key_list=['OS','MYC IHC', 'BCL2 IHC', 'BCL6 IHC', 'CD10 IHC', 'MUM1 IHC'], nclasses=2)
else:
    df_labels = get_pids_labels_for_key(df, key ='OS', nclasses=3)

df_labels

# Graph construction
PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500

# # save paths
# featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
# featpaths = [x if "/0.dat" not in x for x in featpaths]
featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
featpaths = [x for x in featpaths if ("/0.dat" not in x) and ("/file_map.dat" not in x)]
featpaths
pids = [int(osp.basename(featpath).split('_')[0]) for featpath in featpaths]
df_featpaths = pd.DataFrame(zip(pids, featpaths), columns=['patient_id', 'featpath'])

# merge to find datapoints with graph data and labels
df_data = df_featpaths.merge(df_labels, on='patient_id')
# df_data = df_data[:12]

featpaths_data = df_data['featpath'].to_list()
# labels_data = df_data['OS_class'].to_list()
labels_data = df_data[['OS_class','MYC IHC_class', 'BCL2 IHC_class', 'BCL6 IHC_class', 'CD10 IHC_class', 'MUM1 IHC_class']].to_numpy().tolist()
display(labels_data[:5])


[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 0.0, 1.0, 0.0, 0.0, 0.0]]

In [28]:


outgraphpaths_data = [f"{GRAPHSDIR}/{osp.basename(featpath).split('.')[0]}.json" for featpath in featpaths_data]

# save labels
labels_dict = {osp.basename(graphpath): label for graphpath, label in zip(outgraphpaths_data, labels_data)}
with open(LABELSPATH, 'w') as f:
    json.dump(labels_dict, f)

# read normalizer stats from file and pass to fn
dd = np.load(FEATSCALERPATH)
cell_feat_norm_stats = (dd['mean'], dd['var'])

# create final graphs data
shutil.rmtree(GRAPHSDIR)
if not osp.exists(GRAPHSDIR):
    os.makedirs(GRAPHSDIR)
    create_graph_with_pooled_patch_nodes(
        featpaths_data,
        labels_data,
        outgraphpaths_data,
        PATCH_SIZE,
        cell_feat_norm_stats=cell_feat_norm_stats,
        MIN_CELLS_PER_PATCH= MIN_CELLS_PER_PATCH,
        CONNECTIVITY_DISTANCE = CONNECTIVITY_DISTANCE
    )
    # wont work in parallel mode
    # print(np.mean(global_patch_stats), np.std(global_patch_stats))

  0%|          | 0/319 [00:00<?, ?it/s]

|2023-11-22|18:36:50.734| [WARNING] /home/amrit/anaconda3/envs/DL2/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()

|2023-11-22|18:36:50.735| [WARNING] /home/amrit/anaconda3/envs/DL2/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()

|202

# Create data splits

In [17]:
wsi_paths = recur_find_ext(GRAPHSDIR, [".json"])
wsi_names = [Path(v).stem for v in wsi_paths]
assert len(wsi_paths) > 0, "No files found."  # noqa: S101

len(wsi_paths) , len(wsi_names) , wsi_names[:5]

(295,
 295,
 ['13901_HE_1535',
  '13901_HE_1572',
  '13902_HE_1523',
  '13902_HE_1534',
  '13903_HE_1554'])

In [18]:
from src.dset import multilabel_stratified_split

NUM_FOLDS = 1
TEST_RATIO = 0.2
TRAIN_RATIO = 0.8 * 0.7
VALID_RATIO = 0.8 * 0.3

# if SPLIT_PATH and os.path.exists(SPLIT_PATH):
#     splits = joblib.load(SPLIT_PATH)
# else:
x = np.array(wsi_names)
with open(LABELSPATH, 'r') as f:
    labels_dict = json.load(f)
print(labels_dict)
y = np.array([labels_dict[wsi_name+'.json'] for wsi_name in wsi_names])
# y[np.where(y==-1)] = 0
if LABEL_TYPE == "multilabel":
    splits = multilabel_stratified_split(x, y, TRAIN_RATIO, VALID_RATIO, TEST_RATIO, NUM_FOLDS)
else:
    splits = stratified_split(x, y, TRAIN_RATIO, VALID_RATIO, TEST_RATIO, NUM_FOLDS)

joblib.dump(splits, SPLIT_PATH)


{'13901_HE_1535.json': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '13901_HE_1572.json': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0], '13902_HE_1523.json': [1.0, 0.0, 0.0, 1.0, 0.0, 1.0], '13902_HE_1534.json': [1.0, 0.0, 0.0, 1.0, 0.0, 1.0], '13903_HE_1554.json': [1.0, 0.0, 1.0, 0.0, 0.0, 0.0], '13903_HE_1557.json': [1.0, 0.0, 1.0, 0.0, 0.0, 0.0], '13904_HE_1533.json': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], '13904_HE_1569.json': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], '13906_HE_1548.json': [1.0, 0.0, 1.0, 0.0, 0.0, 0.0], '13908_HE_1563.json': [1.0, 0.0, 1.0, 0.0, 0.0, 0.0], '13908_HE_1571.json': [1.0, 0.0, 1.0, 0.0, 0.0, 0.0], '13911_HE_1553.json': [1.0, 0.0, 0.0, 0.0, 0.0, 1.0], '13911_HE_1559.json': [1.0, 0.0, 0.0, 0.0, 0.0, 1.0], '13912_HE_1538.json': [0.0, 0.0, 0.0, 1.0, 1.0, 1.0], '13913_HE_1537.json': [0.0, 1.0, 0.0, 1.0, 0.0, 0.0], '13913_HE_1551.json': [0.0, 1.0, 0.0, 1.0, 0.0, 0.0], '13914_HE_1540.json': [0.0, 0.0, 1.0, 0.0, 0.0, 1.0], '13914_HE_1568.json': [0.0, 0.0, 1.0, 0.0, 0.0, 1.0], '13915_HE_1520.json': [1.0,

['/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/training/splits_HE.dat']

# Look at samples

In [19]:
from src.utils import load_json, rm_n_mkdir, mkdir, recur_find_ext
from src.dset import SlideGraphDataset, stratified_split, StratifiedSampler
import warnings
warnings.filterwarnings("ignore")

from torch_geometric.loader import DataLoader

nodes_preproc_func = None

def sample_data(  # noqa: C901, PLR0912, PLR0915
    dataset_dict: dict,
    GRAPH_DIR = None) -> list:
    
    for subset_name, subset in dataset_dict.items():
        ds = SlideGraphDataset(subset, mode="train", preproc=nodes_preproc_func, graph_dir=GRAPH_DIR)

        batch_sampler = None

        _loader_kwargs = {
            "num_workers": 6,
            "batch_size": 32,
        }
        # # arch_kwargs = {

        # print("subset" ,subset_name , len(subset))
        
        # batch_sampler = batch_sampler = StratifiedSampler(
        #             labels=[v[1] for v in subset],
        #             batch_size=32,
        #         )

        loader =  DataLoader(ds,
                    batch_sampler=batch_sampler,
            drop_last=subset_name == "train" and batch_sampler is None,
            shuffle=subset_name == "train" and batch_sampler is None,
            **_loader_kwargs,)
        
    return ds, loader


for split_idx, split in enumerate(splits):
    new_split = {
                "train": split["train"],
                "infer-train": split["train"],
                "infer-valid-A": split["valid"],
                "infer-valid-B": split["test"],
            }
    ds, loader = sample_data(new_split, GRAPHSDIR)

sample = ds.__getitem__(3)
display(sample)

for _step, batch_data in enumerate(loader):
    print(batch_data)
    break

{'graph': Data(x=[89, 128], edge_index=[2, 472], y=[6], coordinates=[89, 2]),
 'label': tensor(1., dtype=torch.float64)}

{'graph': DataBatch(x=[2235, 128], edge_index=[2, 11622], y=[192], coordinates=[2235, 2], batch=[2235], ptr=[33]), 'label': tensor([1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.],
       dtype=torch.float64)}


In [ ]:
# from src.model import SlideGraphArch

# arch_kwargs = {
#     "dim_features": NUM_NODE_FEATURES,
#     "dim_target": 3,
#     "layers": [64, 32, 32],
#     "dropout": 0.3,
#     "pooling": "mean",
#     "conv": "EdgeConv",
#     "aggr": "max",
#     "CLASSIFICATION_TYPE" : LABEL_TYPE
# }

# model = SlideGraphArch(**arch_kwargs)

# for _step, batch_data in enumerate(tqdm(loader, disable=True)):
#     device = select_device(on_gpu=False)
#     wsi_graphs = batch_data["graph"].to(device)
#     model = model.to(device)

#     # Data type conversion
#     wsi_graphs.x = wsi_graphs.x.type(torch.float32)

#     # Inference mode
#     model.eval()
#     # Do not compute the gradient (not training)
#     with torch.inference_mode():
#         wsi_output, _ = model(wsi_graphs)
#         # print("wsi_output" ,wsi_output)
#     # print("xxx")
#     break
    
# # pred = torch.nn.functional.softmax(wsi_output.type(torch.float32))
# # gt = batch_data['label'].type(torch.float32)
# # display(pred.shape , gt.shape)
# # # criterion = torch.nn.BCELoss()
# # # criterion = torch.nn.()

# # loss = torch.nn.functional.cross_entropy(pred, gt)
# # loss

# Train model

In [10]:
# # we must define the function after training/loading
# def nodes_preproc_func(node_features: np.ndarray) -> np.ndarray:
#     """Pre-processing function for nodes."""
#     return node_scaler.transform(node_features)
nodes_preproc_func = None


splits = joblib.load(SPLIT_PATH)
loader_kwargs = {
    "num_workers": 6,
    "batch_size": 32,
}
# arch_kwargs = {
#     "dim_features": NUM_NODE_FEATURES,
#     "dim_target": NCLASSES,
#     "layers": [32, 32, 16, 8],
#     "dropout": 0.3,
#     "pooling": "mean",
#     "conv": "EdgeConv",
#     "aggr": "max",
# }

if LABEL_TYPE == "OS":
    NCLASSES = 3
else:
    NCLASSES = 6
    
conv = "EdgeConv"
pooling = "mean"
aggr = "max"
dropout= 0.15
layers = [64, 32, 32]

arch_kwargs = {
        "dim_features": NUM_NODE_FEATURES,
        "dim_target": NCLASSES,
        "layers": layers,
        "dropout": dropout,
        "pooling": pooling,
        "conv": conv,
        "aggr": aggr,
        "CLASSIFICATION_TYPE" : LABEL_TYPE
}

RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{STAIN}_{conv}_{pooling}_{aggr}_{str(dropout)}_{str(layers)}_{datetime.now().strftime('%m_%d_%H_%M')}"
RUN_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR = RUN_OUTPUT_DIR / "model"

optim_kwargs = {
    "lr": 5.0e-4,
    "weight_decay": 1.0e-4,
}

NUM_EPOCHS = 100
# if not MODEL_DIR.exists() or True:
for split_idx, split in enumerate(splits):
        new_split = {
            "train": split["train"],
            "infer-train": split["train"],
            "infer-valid-A": split["valid"],
            "infer-valid-B": split["test"],
        }
        MODEL_DIR = Path(MODEL_DIR) 
        split_save_dir = MODEL_DIR / f"{split_idx:02d}/"
        rm_n_mkdir(split_save_dir)
        reset_logging(split_save_dir)
        run_once(new_split,
            NUM_EPOCHS,
            save_dir=split_save_dir,
            arch_kwargs=arch_kwargs,
            loader_kwargs=loader_kwargs,
            optim_kwargs=optim_kwargs,
            on_gpu=ON_GPU,
            GRAPH_DIR=GRAPHSDIR,
            LABEL_TYPE = LABEL_TYPE)

|2023-12-05|15:42:22.539| [INFO] EPOCH: 000


SlideGraphArch(
  (first_h): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (nns): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (convs): ModuleList(
    (0): EdgeConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    ))
    (1): EdgeConv(nn=Sequential(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05,

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:00<00:00, 14.30it/s]
|2023-12-05|15:42:22.611| [INFO] train-EMA-loss: 1.029023
|2023-12-05|15:42:22.940| [INFO] infer-train-accuracy: 0.587100
|2023-12-05|15:42:23.224| [INFO] infer-valid-A-accuracy: 0.623400
|2023-12-05|15:42:23.471| [INFO] infer-valid-B-accuracy: 0.617200
|2023-12-05|15:42:23.696| [INFO] EPOCH: 001


best_score {'infer-train-accuracy': 0, 'infer-valid-A-accuracy': 0, 'infer-valid-B-accuracy': 0}


...

 20%|██        | 1/5 [00:00<00:00, 17.71it/s]
|2023-12-05|15:42:52.466| [INFO] train-EMA-loss: 0.840202
|2023-12-05|15:42:52.780| [INFO] infer-train-accuracy: 0.590800
|2023-12-05|15:42:53.039| [INFO] infer-valid-A-accuracy: 0.632000
|2023-12-05|15:42:53.254| [INFO] infer-valid-B-accuracy: 0.617200
|2023-12-05|15:42:53.487| [INFO] EPOCH: 027
 20%|██        | 1/5 [00:00<00:00, 18.35it/s]
|2023-12-05|15:42:53.544| [INFO] train-EMA-loss: 0.845537
|2023-12-05|15:42:53.837| [INFO] infer-train-accuracy: 0.589900
|2023-12-05|15:42:54.089| [INFO] infer-valid-A-accuracy: 0.627700
|2023-12-05|15:42:54.325| [INFO] infer-valid-B-accuracy: 0.619800
|2023-12-05|15:42:54.570| [INFO] EPOCH: 028
 20%|██        | 1/5 [00:00<00:00, 17.71it/s]
|2023-12-05|15:42:54.629| [INFO] train-EMA-loss: 0.841876
|2023-12-05|15:42:54.921| [INFO] infer-train-accuracy: 0.589900
|2023-12-05|15:42:55.152| [INFO] infer-valid-A-accuracy: 0.625500
|2023-12-05|15:42:55.405| [INFO] infer-valid-B-accuracy: 0.619800
|2023-12-05|

best_score {'infer-train-accuracy': 0.618, 'infer-valid-A-accuracy': 0.6385, 'infer-valid-B-accuracy': 0.6328, 'infer-train-label_accuracies': [0.5393, 0.8483, 0.5225, 0.6854, 0.618, 0.4944], 'infer-train-best_epoch': 47, 'infer-valid-A-label_accuracies': [0.5455, 0.8442, 0.5195, 0.7792, 0.6104, 0.5325], 'infer-valid-A-best_epoch': 50, 'infer-valid-B-label_accuracies': [0.4844, 0.875, 0.5469, 0.7031, 0.7344, 0.4531], 'infer-valid-B-best_epoch': 44}


 20%|██        | 1/5 [00:00<00:00, 17.48it/s]
|2023-12-05|15:43:21.539| [INFO] train-EMA-loss: 0.755678
|2023-12-05|15:43:21.859| [INFO] infer-train-accuracy: 0.618900
|2023-12-05|15:43:22.116| [INFO] infer-valid-A-accuracy: 0.642900
|2023-12-05|15:43:22.404| [INFO] infer-valid-B-accuracy: 0.625000
|2023-12-05|15:43:22.656| [INFO] EPOCH: 052
 20%|██        | 1/5 [00:00<00:00, 17.21it/s]
|2023-12-05|15:43:22.716| [INFO] train-EMA-loss: 0.762301
|2023-12-05|15:43:23.017| [INFO] infer-train-accuracy: 0.624500
|2023-12-05|15:43:23.271| [INFO] infer-valid-A-accuracy: 0.658000
|2023-12-05|15:43:23.532| [INFO] infer-valid-B-accuracy: 0.627600
|2023-12-05|15:43:23.783| [INFO] EPOCH: 053
 20%|██        | 1/5 [00:00<00:00, 17.64it/s]
|2023-12-05|15:43:23.841| [INFO] train-EMA-loss: 0.755612
|2023-12-05|15:43:24.166| [INFO] infer-train-accuracy: 0.627300
|2023-12-05|15:43:24.412| [INFO] infer-valid-A-accuracy: 0.655800
|2023-12-05|15:43:24.692| [INFO] infer-valid-B-accuracy: 0.632800
|2023-12-05|

best_score {'infer-train-accuracy': 0.6461, 'infer-valid-A-accuracy': 0.6624, 'infer-valid-B-accuracy': 0.6328, 'infer-train-label_accuracies': [0.5618, 0.8483, 0.573, 0.6854, 0.6517, 0.5562], 'infer-train-best_epoch': 75, 'infer-valid-A-label_accuracies': [0.5325, 0.8442, 0.5844, 0.7792, 0.6364, 0.5974], 'infer-valid-A-best_epoch': 61, 'infer-valid-B-label_accuracies': [0.4375, 0.875, 0.5625, 0.7031, 0.6875, 0.5312], 'infer-valid-B-best_epoch': 61}


 20%|██        | 1/5 [00:00<00:00, 16.68it/s]
|2023-12-05|15:43:51.197| [INFO] train-EMA-loss: 0.691174
|2023-12-05|15:43:51.519| [INFO] infer-train-accuracy: 0.641400
|2023-12-05|15:43:51.791| [INFO] infer-valid-A-accuracy: 0.655800
|2023-12-05|15:43:52.081| [INFO] infer-valid-B-accuracy: 0.609400
|2023-12-05|15:43:52.374| [INFO] EPOCH: 077
 20%|██        | 1/5 [00:00<00:00, 17.08it/s]
|2023-12-05|15:43:52.435| [INFO] train-EMA-loss: 0.688605
|2023-12-05|15:43:52.765| [INFO] infer-train-accuracy: 0.642300
|2023-12-05|15:43:53.032| [INFO] infer-valid-A-accuracy: 0.651500
|2023-12-05|15:43:53.311| [INFO] infer-valid-B-accuracy: 0.619800
|2023-12-05|15:43:53.591| [INFO] EPOCH: 078
 20%|██        | 1/5 [00:00<00:00, 16.52it/s]
|2023-12-05|15:43:53.654| [INFO] train-EMA-loss: 0.697659
|2023-12-05|15:43:53.991| [INFO] infer-train-accuracy: 0.641400
|2023-12-05|15:43:54.252| [INFO] infer-valid-A-accuracy: 0.649400
|2023-12-05|15:43:54.529| [INFO] infer-valid-B-accuracy: 0.612000
|2023-12-05|

# Accuracy metrics

In [ ]:
pred = [[1.46648765e+00,-8.77613187e-01,-2.51502216e-01,-1.10481453e+00
,6.64071560e-01,-5.99045038e-01]
,[-1.80002856e+00,8.66671383e-01,-6.64208949e-01,1.20571077e+00
,-2.28768730e+00,1.23205519e+00]
,[-8.38510573e-01,2.11250722e-01,4.82147932e-03,5.65632761e-01
,-1.94593978e+00,6.11969411e-01]
,[1.70954013e+00,-7.06104040e-01,-6.74402267e-02,-8.06845903e-01
,1.92917514e+00,-1.58036017e+00]
,[1.23513281e+00,-6.98642850e-01,-7.90928781e-01,-6.50385141e-01
,9.64580655e-01,-5.73552132e-01]
,[-9.15723860e-01,3.66611242e-01,3.33443969e-01,4.22663540e-01
,-7.69037247e-01,1.99665681e-01]]

gt = [[1.,1.,0.,0.,0.,0.]
,[0.,0.,0.,0.,0.,1.]
,[1.,1.,0.,1.,1.,0.]
,[1.,0.,0.,0.,0.,0.]
,[0.,1.,1.,0.,0.,1.]
,[1.,1.,1.,1.,1.,0.]]


def multilabel_accuracy(y_pred, y_true):
    """
    Calculate multilabel classification metrics.

    Parameters:
    - y_true: 2D array-like or binary indicator matrix, true labels
    - y_pred: 2D array-like or binary indicator matrix, predicted labels

    Returns:
    - overall_accuracy: Subset accuracy (exact match ratio)
    - label_accuracies: List of accuracies for each individual label
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred[y_pred > 0.5] = 1
    y_pred[y_pred < 0.5] = 0

    # Ensure the shapes are the same
    if y_true.shape != y_pred.shape:
        raise ValueError("Shapes of y_true and y_pred must be the same.")
    
    label_accuracies = np.mean(y_true == y_pred, axis=0).round(4)
    overall_accuracy = np.mean(label_accuracies).round(4)

    return overall_accuracy, label_accuracies

multilabel_accuracy(pred, gt)